### Acknowledgement:
Most of the code in this notebook is taken from [OpenMC Documentation >> Examples >> Modeling a Pincell](https://nbviewer.org/github/openmc-dev/openmc-notebooks/blob/main/pincell.ipynb)

### Other Notes:
- export_to_xml() functions have been commented out, you need to uncomment them in order to run the code
- openmc.run() has also been commented out, uncomment it to run the code
- A lot of the tutorial at the above link has been removed. It is really helpful though, and I encourage you to follow the link and look through that example yourself at some point

In [1]:
import openmc

## Materials

In [2]:
uo2 = openmc.Material(name="uo2") # name material
uo2.add_nuclide('U235', 0.03) # add nuclides (default percent type is 'ao' (atom percent))
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0) # set density
uo2.temperature = 293

In [3]:
zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)
zirconium.temperature = 293

In [4]:
water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.temperature = 293
water.add_s_alpha_beta('c_H_in_H2O') # assign tsl

In [5]:
materials = openmc.Materials([uo2, zirconium, water])

In [33]:
# materials.export_to_xml()

## Geometry

In [20]:
# concentric cylinders for pincell
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

In [21]:
# creates regions of concentric cylinders for pincell
fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

In [22]:
# fuel cell
fuel = openmc.Cell(name='fuel')
fuel.fill = uo2
fuel.region = fuel_region

# gap cell
gap = openmc.Cell(name='air gap')
gap.region = gap_region
# note: if you don't specify a fill, it automatically makes that region empty

# clad cell
clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

In [23]:
pitch = 1.26 # sets the dimensions for the outer box of the pincell

# creates individual planes for outer box
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
bottom = openmc.YPlane(-pitch/2, boundary_type='reflective')
top = openmc.YPlane(pitch/2, boundary_type='reflective')

In [24]:
# create the region with water that's outside the clad and inside the bounding box
water_region = +left & -right & +bottom & -top & +clad_outer_radius

# creates the cell for water
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

In [25]:
root_universe = openmc.Universe(cells=(fuel, gap, clad, moderator))

geometry = openmc.Geometry()
geometry.root_universe = root_universe

In [26]:
# root_universe.plot(width=(1.2, 1.2), basis='xy')

In [34]:
# geometry.export_to_xml()

## Settings

In [28]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.IndependentSource(space=point)

In [29]:
settings = openmc.Settings()
settings.source = source
settings.batches = 150
settings.inactive = 20
settings.particles = 1000

In [35]:
# settings.export_to_xml()

## Run OpenMC

In [36]:
# now in order to run the file simply use the function openmc.run()
# openmc.run()